In [ ]:
using QuantumOptics
using LinearAlgebra
using SparseArrays

# The Boson Fock Space 

In [ ]:
max_occupation_number = 2
occ_array = []
for i in 0:max_occupation_number
    push!(occ_array, i)
end

In [ ]:
Nx = Ny = 5
N = Nx*Ny 
b = NLevelBasis(N)

In [ ]:
# states = bosonstates(b, [occ_array...]) #en az 0 en fazla 2 bozon
states = bosonstates(b, [2])
for i in 1:length(states)
    println("$i: ", states[i])
end

In [ ]:
b_mb_b = ManyBodyBasis(b, states)

# The Periodic Lattice 

In [1]:
using OffsetArrays
lat = range(1,Nx*Ny)
latt = reshape(lat, (Nx,Ny))
lattice = OffsetArray(latt, 0:Nx-1, 0:Ny-1)
x_co = range(0, Nx-1) 
y_co = range(0, Ny-1)
arr = []
for i in 1:length(x_co)
    for j in 1:length(y_co)
       arr = [arr;x_co[i];y_co[j]]
    end
end
xy = reshape(arr, (2,Nx*Ny)) |> transpose
arr = OffsetArray(arr,0:Nx*Ny*2-1)
lattice

In [ ]:
x = []
function PerBC()
    for i in 0:Nx-1
        for j in 0:Ny-1
            cc = [lattice[mod(j,Ny),mod(i-1,Nx)],lattice[mod(j+1,Ny),mod(i,Nx)],lattice[mod(j,Ny),mod(i+1,Nx)],lattice[mod(j-1,Ny),mod(i,Nx)]]
            push!(x,cc)
        end
    end
    return x
end

# Hopping Phase Values

In [2]:
function HP(m, n, alpha)
    if abs(xy[m,1]-xy[n,1])==Nx-1
        if xy[m,1] > xy[n,1]
            A = -exp(-1im*2*pi*alpha*xy[m,2])
            #println(m," -> ",n," Kenarda x+ ve faz: ", A)
        elseif xy[m,1] < xy[n,1]
            A = -exp(1im*2*pi*alpha*xy[m,2])
            #println(m," -> ",n," Kenarda x- ve faz: ", A)
        end
    elseif m==n
        A = 0
        #println(m, " -> ", n, " Aynı noktalar ve faz: ", A)
    else
        if xy[m,1] > xy[n,1]
            A = -exp(1im*2*pi*alpha*xy[m,2])
            #println(m," -> ",n," x- ve faz: ", A)
        elseif xy[m,1] < xy[n,1]
            A = -exp(-1im*2*pi*alpha*xy[m,2])
            #println(m," -> ",n," x+ ve faz: ", A)
        else
            A = -exp(0)
            #println(m," -> ",n," x sabit ve faz: ", A)
        end
    end
end 

# Many Body Hamiltonian

In [ ]:
alpha = 1/5
S = SparseOperator(b_mb_b)
for m in 1:N
    for n in 1:N
        if m in PerBC()[n]
            S = S + HP(m, n, alpha)*transition(b_mb_b, m, n)
        end
    end
end
eigenenergies(dense(S))

# Hopping Phase Values with Twisted Angle Parameters

In [ ]:
Tsize=20
dx=2*pi/Tsize
dy=dx
Tx=range(start=0, stop=2*pi, step=dx) #şimdilik 1'den başlatıyorum. Chern yanlış hesaplarsam burayla ilgilenicem(!)
Ty=range(start=0, stop=2*pi, step=dy) 

In [ ]:
q=Nx
ChernAlpha=1/q
function HPTA(m, n, Tx, Ty, ChernAlpha)
    if abs(xy[m,1]-xy[n,1])==Nx-1
        if xy[m,1] > xy[n,1]
            B = -exp(-1im*2*pi*ChernAlpha*xy[m,2])*exp(-1im*Tx)
        elseif xy[m,1] < xy[n,1]
            B = -exp(1im*2*pi*ChernAlpha*xy[m,2])*exp(1im*Tx)
        end
    elseif abs(xy[m,2]-xy[n,2])==Ny-1
        if xy[m,2] > xy[n,2]
            B = -exp(-1im*Ty)
        elseif xy[m,2] < xy[n,2]
            B = -exp(1im*Ty)
        end
    elseif m==n
        B = 0
    else
        if xy[m,1] > xy[n,1]
            B = -exp(1im*2*pi*ChernAlpha*xy[m,2])
        elseif xy[m,1] < xy[n,1]
            B = -exp(-1im*2*pi*ChernAlpha*xy[m,2])
        else
            B = -exp(0)
        end
    end
end 

# Many Body Chern Calculations

# E'ler D-Manifold Dejenere Olması Lazım
Örneğin q=2 ve parçacık sayısı=2 iken 10 durum var ve 0:5,5:10 dejenere bekliyorum.
veya q=3 ve parçacık sayısı =2 iken 45 durum var ve 0:15,15:30,30:45 dejenere bekliyorum. Herhangi theta değeri için bu durumun korunması lazım.
Twist fazları doğru ekleniyor bunu real space hesabında gördük.

Çıktılar: q=3, 325 durum için dejenerelik 1:15...

In [ ]:
function HMTA(ChernAlpha, Tx, Ty)
    STA = SparseOperator(b_mb_b)
    for m in 1:N
        for n in 1:N
            if m in PerBC()[n]
                STA = STA + HPTA(m, n, Tx, Ty, ChernAlpha) * transition(b_mb_b, m, n)
            end
        end
    end
    return STA
end
println("Eigenenergies:")
eigenenergies(dense(HMTA(ChernAlpha, 9,1)))[1:15]

In [ ]:
println("Eigenstates:")
eigenstates(dense(HMTA(ChernAlpha, 1,1)))[2]

In [ ]:
function HMTA(ChernAlpha, Tx, Ty)
    STA = SparseOperator(b_mb_b)
    for m in 1:N
        for n in 1:N
            if m in PerBC()[n]
                STA = STA + HPTA(m, n, Tx, Ty, ChernAlpha)*transition(b_mb_b, m, n)
            end
        end
    end
    return STA
end
#------------------------
#PROBLEM: QOJulia kütüphanesi öz-vektörleri kare matris şeklinde depolamıyor!
#------------------------
#ÇÖZÜM(Çok Yavaş):
function EigVEC(ChernAlpha, Tx, Ty)
    EigVec = spzeros(Complex{Float64},length(states),length(states))
    for m in 1:length(states)
        for n in 1:length(states)
            EigVec[m,n]=eigenstates(dense(HMTA(ChernAlpha, Tx, Ty)))[2][m].data[n]
        end
    end
    return EigVec
end
EigVEC(ChernAlpha, 2, 3)

In [ ]:
ChernArray = zeros(Complex{Float64},0)
for i in range(start=1, stop=Nx*Ny, step=q) 
    j=i+q
    Sum=0
    n1=i
    n2=j
    for tx in range(start=1, stop=length(Tx))
        for ty in range(start=1, stop=length(Ty))
            w1=eigenstates(dense(HMTA(ChernAlpha, Tx[tx], Ty[ty])))[1]
            v1=EigVEC(ChernAlpha, Tx[tx], Ty[ty])
            idx1 = sortperm(w1)
            v1sorted = v1[:,idx1]
            v11 = v1sorted[:,n1:n2]
            #------------------------------------
            w2=eigenstates(dense(HMTA(ChernAlpha, Tx[tx]+dx, Ty[ty])))[1]
            v2=EigVEC(ChernAlpha, Tx[tx]+dx, Ty[ty])
            idx2 = sortperm(w2)
            v2sorted = v2[:,idx2]
            v22 = v2sorted[:,n1:n2]
            #------------------------------------
            w3=eigenstates(dense(HMTA(ChernAlpha, Tx[tx], Ty[ty]+dy)))[1]
            v3=EigVEC(ChernAlpha, Tx[tx], Ty[ty]+dy)
            idx3 = sortperm(w3)
            v3sorted = v3[:,idx3]
            v33 = v3sorted[:,n1:n2]
            #------------------------------------
            w4=eigenstates(dense(HMTA(ChernAlpha, Tx[tx]+dx, Ty[ty]+dy)))[1]
            v4=EigVEC(ChernAlpha, Tx[tx]+dx, Ty[ty]+dy)
            idx4 = sortperm(w4)
            v4sorted = v4[:,idx4]
            v44 = v4sorted[:,n1:n2]
            #----------LINK VARIABLES------------
            U1=det(adjoint(v11)*v22)
            U1=U1/abs(U1)
            U2=det(adjoint(v22)*v44)
            U2=U2/abs(U2)
            U3=det(adjoint(v33)*v44)
            U3=U3/abs(U3)
            U4=det(adjoint(v11)*v33)
            U4=U4/abs(U4)
            #----------BERRY CURVATURE-----------
            F=log(U1*U2*1/U3*1/U4)
            Sum=Sum+F
        end
    end
    Chern=1/(2*pi*1im)*Sum
    append!(ChernArray, real(Chern))
end
ChernArray